Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [2]:
using Plots, ApproxFun

In [3]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [4]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [5]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [6]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.1102230246251565e-16 - 0.26835469053190497im
-2.2545984176836065e-16 + 0.26835469053190486im
0.03747531973228924 + 0.999999999999999im
-0.037475319732288855 + 0.9999999999999999im

Floquet Multipliers = 

0.6651802570066612 - 0.7466828146465885im
0.6651802570066612 + 0.746682814646588im
-1.1249427991479368 + 3.634790471420913e-15im
-0.8889340869219565 + 5.036290646901535e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [7]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [8]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,273.821 596.059,273.868 596.05,273.962 596.036,274.103 596.018,274.291 595.996,274.526 595.969,274.808 595.938,275.136 595.903,275.51 595.862,275.931 
 595.818,276.398 595.769,276.91 595.715,277.468 595.657,278.071 595.595,278.718 595.528,279.41 595.457,280.146 595.381,280.926 595.301,281.748 595.217,282.613 
 595.128,283.521 595.034,284.469 594.936,285.458 594.834,286.488 594.727,287.556 594.616,288.664 594.5,289.809 594.38,290.991 594.255,292.209 594.126,293.463 
 593.993,294.75 593.855,296.071 593.713,297.424 593.566,298.807 593.415,300.22 593.26,301.662 593.1,303.13 592.936,304.624 592.767,306.142 592.594,307.683 
 592.416,309.245 592.234,310.826 592.048,312.425 591.857,314.039 591.662,315.668 591.463,317.308 591.259,318.959 591.051,320.617 590.838,322.282 590.621,323.949 
 590.4,325.618 590.174,327.286 589.944,328.951 589.709,330.609 589.47,332.259 589.227,333.897 588.98,335.522 588.728,337.13 588.472,338.718 588.211,340.284 
 587.946,341.825 587.677,343.337 587.404,344.818 587.126,346.265 586.844,347.673 586.557,349.042 586.266,350.366 585.971,351.643 585.672,352.87 585.368,354.043 
 585.06,355.159 584.748,356.215 584.432,357.207 584.111,358.132 583.786,358.988 583.457,359.77 583.123,360.476 582.785,361.102 582.443,361.646 582.097,362.103 
 581.746,362.472 581.392,362.749 581.033,362.931 580.67,363.015 580.302,363 579.931,362.882 579.555,362.658 579.175,362.327 578.791,361.885 578.402,361.332 
 578.01,360.665 577.613,359.881 577.212,358.98 576.807,357.96 576.398,356.82 575.985,355.558 575.567,354.173 575.146,352.665 574.72,351.032 574.29,349.275 
 573.856,347.393 573.418,345.387 572.976,343.256 572.53,341 572.08,338.621 571.626,336.119 571.167,333.496 570.705,330.752 570.238,327.889 569.768,324.909 
 569.293,321.813 568.814,318.605 568.332,315.286 567.845,311.86 567.354,308.328 566.86,304.695 566.361,300.963 565.858,297.136 565.352,293.219 564.841,289.215 
 564.327,285.128 563.808,280.963 563.286,276.725 562.76,272.418 562.229,268.049 561.695,263.622 561.157,259.142 560.615,254.617 560.069,250.051 559.519,245.451 
 558.966,240.824 558.408,236.175 557.847,231.511 557.282,226.84 556.713,222.168 556.14,217.502 555.563,212.849 554.983,208.217 554.398,203.613 553.81,199.043 
 553.218,194.516 552.623,190.039 552.023,185.619 551.42,181.262 550.813,176.977 550.203,172.769 549.588,168.647 548.97,164.615 548.349,160.681 547.723,156.851 
 547.094,153.13 546.461,149.524 545.825,146.038 545.185,142.676 544.541,139.443 543.894,136.342 543.243,133.376 542.588,130.549 541.93,127.862 541.268,125.317 
 540.603,122.915 539.934,120.656 539.261,118.541 538.585,116.567 537.905,114.734 537.222,113.04 536.536,111.482 535.845,110.056 535.152,108.76 534.455,107.588 
 533.754,106.536 533.05,105.599 532.343,104.771 531.632,104.046 530.917,103.418 530.199,102.88 529.478,102.426 528.754,102.048 528.026,101.741 527.295,101.496 
 526.56,101.308 525.822,101.17 525.08,101.076 524.336,101.019 523.588,100.995 522.837,100.997 522.082,101.023 521.324,101.066 520.563,101.126 519.799,101.198 
 519.031,101.281 518.26,101.374 517.486,101.478 516.709,101.592 515.929,101.72 515.145,101.863 514.359,102.026 513.569,102.213 512.776,102.43 511.98,102.684 
 511.18,102.983 510.378,103.335 509.573,103.75 508.764,104.239 507.953,104.812 507.138,105.483 506.32,106.263 505.5,107.165 504.676,108.205 503.849,109.394 
 503.02,110.748 502.187,112.28 501.352,114.006 500.513,115.937 499.672,118.089 498.827,120.473 497.98,123.101 497.13,125.983 496.277,129.131 495.421,132.55 
 494.562,136.249 493.701,140.232 492.837,144.502 491.969,149.058 491.099,153.901 490.227,159.025 489.351,164.425 488.473,170.091 487.592,176.012 486.708,182.175 
 485.822,188.562 484.933,195.155 484.041,201.932 483.147,208.87 482.